In [1]:
import numpy as np
from helper import awgn, draw, hermitian, toep
import random
import cvxpy as cp
epsilon = 1e-8

In [2]:
M = 10 # number of array elem
Kt = 3 # number of signal source
# k := k'th signal source
snr = 20 # db signal to noise
L = 5
f_k = np.array([ random.random() for j in range(Kt)]).reshape(Kt,1)
a = lambda f_k : np.array([ np.exp(1j*2*np.pi*i*f_k) for i in range(M) ], dtype=complex).reshape(M,1)
s_k = np.vstack( [np.array([ random.randint(-10,10)+random.randint(-10,10)*1j for i in range(L) ]).reshape(1,L) for j in range(Kt)] )
c_k = np.array([ np.linalg.norm(s_k[j,:]) for j in range(Kt) ])
b_k = np.vstack([ (s_k[j,:] / c_k[j]).reshape(1,L) for j in range(Kt) ])
for j in range(Kt):
    try:
        assert np.linalg.norm(c_k[j]) > 0
        assert np.abs(np.linalg.norm(b_k[j,:]) - 1) < epsilon
    except:
        print(c_k[j])
        print(np.linalg.norm(b_k[j,:]))
        assert False
X = np.zeros((M,1))
for j in range(Kt):
    X = X + c_k[j] * (a(f_k[j]) * b_k[j,:].reshape(M,1))
Y = X + awgn(X, snr)

ValueError: cannot reshape array of size 5 into shape (10,1)

In [ ]:
Ry = np.corrcoef(Y@hermitian(Y), dtype=complex)
print(Ry.shape)

(10, 10)


In [ ]:
# constants
epsilon = 1e-8
# variables
W = cp.Variable((1,1), complex=True)
X = cp.Variable((M,1), complex=True)
u = cp.Variable((M,1), complex=True)
# constraints
matrix = cp.bmat([
    [toep(u), X],
    [    X.H, W]
])
constraint = [ matrix >> 0 ]
constraint += [ cp.norm(Y-X, 'fro')**2 <= epsilon ]
# problem
prob = cp.Problem(cp.Minimize(0.5*( cp.real(cp.trace(W) + cp.trace(toep(u))) )))
prob.solve()
print(prob.status)


unbounded


In [ ]:
print(W.value, X.value, u.value)
print(cp.norm(Y-X.value, 'fro')**2)

None None None


TypeError: unsupported operand type(s) for -: 'complex' and 'NoneType'